In [2]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------

def get_current_weather(city: str, unit: str = "celsius") -> str:
    return f"It is 23°C and sunny in {city}."

In [3]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------

SYSTEM_PROMPT = """
You are an assistant that can use tools.

You have access to the following tool:

Tool name: get_current_weather
Description: Get the current weather for a city.
Arguments:
- city (string): Name of the city
- unit (string, optional): "celsius" or "fahrenheit"

When the user asks about weather, you MUST respond ONLY with:

TOOL_CALL: {"name": "get_current_weather", "args": {"city": "<city>"}}

Do not add any extra text.
"""

USER_QUESTION = "What is the weather in San Diego today?"


In [6]:
from openai import OpenAI

client = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)

# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------

response = client.chat.completions.create(
    model="gemma3:1b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION},
    ],
)

raw_output = response.choices[0].message.content
print("RAW MODEL OUTPUT:")
print(raw_output)


RAW MODEL OUTPUT:
TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}



In [7]:
# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------

import re
import json

match = re.search(r"TOOL_CALL:\s*(\{.*\})", raw_output)

if match:
    tool_call = json.loads(match.group(1))
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    print(f"Calling tool `{tool_name}` with args {tool_args}")

    if tool_name == "get_current_weather":
        result = get_current_weather(**tool_args)
        print("Result:", result)
else:
    print("No tool call detected.")

Calling tool `get_current_weather` with args {'city': 'San Diego'}
Result: It is 23°C and sunny in San Diego.


In [8]:
from pprint import pprint
import inspect

def to_schema(fn):
    sig = inspect.signature(fn)
    params = {}

    for name, param in sig.parameters.items():
        param_type = "string"
        if param.annotation is int:
            param_type = "integer"
        elif param.annotation is float:
            param_type = "number"
        elif param.annotation is bool:
            param_type = "boolean"

        params[name] = {
            "type": param_type,
            "required": param.default is inspect._empty
        }

        if param.default is not inspect._empty:
            params[name]["default"] = param.default

    return {
        "name": fn.__name__,
        "description": fn.__doc__ or "No description provided.",
        "parameters": params
    }


In [9]:
tool_schema = to_schema(get_current_weather)
pprint(tool_schema)


{'description': 'No description provided.',
 'name': 'get_current_weather',
 'parameters': {'city': {'required': True, 'type': 'string'},
                'unit': {'default': 'celsius',
                         'required': False,
                         'type': 'string'}}}


In [10]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)

SYSTEM_PROMPT = """
You are an assistant that can decide when to call tools.

If a tool is needed, respond ONLY in the following format:

TOOL_CALL: {"name": "<tool_name>", "args": {...}}

Do not include any additional text.
"""

TOOL_SPEC_PROMPT = json.dumps(
    {"tools": [tool_schema]},
    indent=2
)

USER_QUESTION = "What is the weather in San Diego today?"


In [11]:
response = client.chat.completions.create(
    model="gemma3:1b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "system", "name": "tool_spec", "content": TOOL_SPEC_PROMPT},
        {"role": "user", "content": USER_QUESTION},
    ],
)

raw_output = response.choices[0].message.content
print("RAW MODEL OUTPUT:")
print(raw_output)


RAW MODEL OUTPUT:
TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}

